In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.datamad0819 

In [12]:
project = {"_id":0}

locations = db.locations.find()
data = pd.DataFrame(locations)
data = data.drop("_id", axis=1)
locations = data.rename(columns={"lat":"Latitude","long":"Longitude"})
display(locations.head())
print(locations.shape)

,name,category_code,city,Latitude,Longitude,position
0,Geni,web,West Hollywood,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Lala,games_video,Palo Alto,37.451151,-122.154369,"{'type': 'Point', 'coordinates': [-122.154369,..."
2,Joost,games_video,New York,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
3,Gizmoz,web,Menlo Park,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
4,eBay,web,San Jose,37.295005,-121.930035,"{'type': 'Point', 'coordinates': [-121.930035,..."


(3428, 6)


In [14]:
def getCoords(lat,long):
    coords = []
    for x,y in zip(lat,long):
        coords.append((x,y))
    return coords

In [26]:
coordinates = getCoords(locations["Latitude"],locations["Longitude"])
print(coordinates[:10])

[(34.090368, -118.393064), (37.451151, -122.154369), (40.7464969, -74.0094471), (37.48413, -122.169472), (37.295005, -121.930035), (37.387845, -122.055197), (37.779558, -122.393041), (40.724604, -73.996876), (38.926172, -77.245195), (34.0263021, -118.380954)]


In [27]:
def getCompaniesNear(lat,long, max_meters=1000):
    x = list(db.locations.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$maxDistance": max_meters
             }
       }
    }))
    return ((lat,long),len(x))

In [31]:
list_coords = []
for x in coordinates:
    y = getCompaniesNear(x[0],x[1])
    list_coords.append(y)

In [36]:
def sort_by_count(arr):
    return sorted(arr,key=lambda x: x[1],reverse=True)

In [37]:
list_coords_final = sort_by_count(list_coords)

In [44]:
def getCompanies(lat,long, max_meters=1000):
    return list(db.locations.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$maxDistance": max_meters
             }
       }
    }))

In [45]:
final_location = getCompanies(*x)

In [48]:
final_location = pd.DataFrame(final_location)

In [50]:
final_location = final_location.drop("_id", axis=1)
final_location = final_location.rename(columns={"lat":"Latitude","long":"Longitude"})
display(final_location.head())
print(final_location.shape)

,name,category_code,city,Latitude,Longitude,position
0,ON24,web,San Francisco,37.785033,-122.400474,"{'type': 'Point', 'coordinates': [-122.4004738..."
1,Neocase Software,software,San Francisco,37.785655,-122.398393,"{'type': 'Point', 'coordinates': [-122.398393,..."
2,CITTIO,software,San Francisco,37.786761,-122.401459,"{'type': 'Point', 'coordinates': [-122.4014585..."
3,Kongregate,games_video,San Francisco,37.786942,-122.401245,"{'type': 'Point', 'coordinates': [-122.401245,..."
4,Playboox,software,San Francisco,37.787028,-122.400019,"{'type': 'Point', 'coordinates': [-122.4000192..."


(139, 6)


In [52]:
final_location.to_json('../data/final_location.json', orient="records")